# Deep Learning mit Keras

Dieses Notebook trainiert mehrere Modell zu den zwei Datensätzen `banknote_authentication.csv` und `spambase.csv`.

Zuerst wird der Datensatz `banknote_authentication.csv` betrachtet.

##  Spam erkennen

**Bibliotheken einbinden und Daten formatieren**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from IPython.display import display, HTML

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.activations import relu, elu, sigmoid
from keras.layers.core import Dense
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers, initializers, callbacks
import talos as ta
from pprint import pprint

Using TensorFlow backend.


## Banknoten Authentifizierung

In [2]:
dat = pd.read_csv("./banknote_authentication.csv")
print(dat.shape)

display(dat.head())
print(dat.count())

(1372, 5)


,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


variance    1372
skewness    1372
curtosis    1372
entropy     1372
class       1372
dtype: int64


In [3]:
train_labels = dat[['class']].values
X = dat.drop(['class'], axis=1).values
train_samples = np.array(X)

Nachdem die Daten importiert wurden müssen nun alle Features skaliert werden.

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train_samples = scaler.fit_transform(train_samples)

In [5]:
p_bank = {'activation':['sigmoid', 'softmax', 'relu'], #'relu', 'elu', 'softmax', 
        'optimizer': ['RMSprop'], #'Nadam', 'Adam', 'sgd', 
        'losses': ['binary_crossentropy'],
        'hidden_layers':[1, 2],
        'first_neurons': [2, 40], # 3, 
        'batch_size': [30, 50],
        'epochs': [600], # 20, 100, 
        'random_state':  [1337]}

In [6]:
def banknote(x_train, y_train, x_val, y_val, params):

        model = Sequential()
        model.add(Dense(params['first_neurons'], input_dim=4, activation=params['activation'],
                       kernel_initializer = initializers.RandomUniform(seed = params['random_state'])))
        
        model.add(Dense(1, activation='sigmoid'))
        
        model.compile(optimizer=params['optimizer'], loss=params['losses'], metrics = ['accuracy'])

        out = model.fit(x_train, y_train,
                         batch_size=params['batch_size'],
                         epochs=params['epochs'],
                         validation_data=[x_val, y_val],
                         verbose=0)

        return out, model

In [7]:
scan = ta.Scan(scaled_train_samples, train_labels, model=banknote, params=p_bank)

  0%|          | 0/24 [00:00<?, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


100%|██████████| 24/24 [04:17<00:00,  9.26s/it]


In [8]:
result = pd.DataFrame(scan.data)
result.head()
result.sort_values(by='acc', ascending=False).head()

,round_epochs,val_loss,val_acc,loss,acc,activation,optimizer,losses,hidden_layers,first_neurons,batch_size,epochs,random_state
21,600,0.043172,0.987864,0.024912,0.997917,softmax,RMSprop,binary_crossentropy,2,40,50,600,1337
15,600,0.041605,0.987864,0.024062,0.997917,softmax,RMSprop,binary_crossentropy,1,40,50,600,1337
12,600,0.029280,0.987864,0.012670,0.997917,softmax,RMSprop,binary_crossentropy,1,40,30,600,1337
10,600,0.029219,0.987864,0.012914,0.997917,softmax,RMSprop,binary_crossentropy,2,40,30,600,1337
14,600,0.022321,0.992718,0.019363,0.994792,relu,RMSprop,binary_crossentropy,1,40,30,600,1337
